# Notebook 14: Occupancy Optimization## Phase 3: Optimization Fundamentals**Learning Objectives:**- Understand occupancy calculation- Learn resource usage- Master performance tuning- Apply concepts in practical scenarios- Measure and analyze performance

## Concept: Occupancy Optimization**Topics Covered:**- occupancy calculation- resource usage- performance tuning**Key Concepts:**This notebook covers occupancy calculation in the context of Phase 3: Optimization Fundamentals.

## Example 1: Basic Occupancy Optimization

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)


template<int BLOCK_SIZE>
__global__ void occupancyTestKernel(float* out, const float* in, int n) {
    __shared__ float smem[BLOCK_SIZE];
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tid = threadIdx.x;

    if (idx < n) smem[tid] = in[idx];
    __syncthreads();

    float val = smem[tid];
    for (int i = 0; i < 50; i++) val = sqrtf(val + 1.0f);

    if (idx < n) out[idx] = val;
}

void testOccupancy(int blockSize, int N) {
    size_t bytes = N * sizeof(float);
    float *h_in = (float*)malloc(bytes);
    for (int i = 0; i < N; i++) h_in[i] = (float)i;

    float *d_in, *d_out;
    CUDA_CHECK(cudaMalloc(&d_in, bytes));
    CUDA_CHECK(cudaMalloc(&d_out, bytes));
    CUDA_CHECK(cudaMemcpy(d_in, h_in, bytes, cudaMemcpyHostToDevice));

    int blocks = (N + blockSize - 1) / blockSize;

    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));
    CUDA_CHECK(cudaEventRecord(start));

    if (blockSize == 128) occupancyTestKernel<128><<<blocks, blockSize>>>(d_out, d_in, N);
    else if (blockSize == 256) occupancyTestKernel<256><<<blocks, blockSize>>>(d_out, d_in, N);
    else if (blockSize == 512) occupancyTestKernel<512><<<blocks, blockSize>>>(d_out, d_in, N);

    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaEventSynchronize(stop));

    float ms;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
    printf("Block size %4d: %.3f ms\n", blockSize, ms);

    free(h_in);
    cudaFree(d_in); cudaFree(d_out);
    cudaEventDestroy(start); cudaEventDestroy(stop);
}

int main() {
    printf("=== Occupancy Tuning ===\n\n");
    const int N = 1 << 24;
    int blockSizes[] = {128, 256, 512};

    for (int i = 0; i < 3; i++)
        testOccupancy(blockSizes[i], N);

    printf("\nOptimal occupancy depends on register and shared memory usage\n");
    return 0;
}


## Practical ExerciseComplete the following exercises to practice the concepts learned.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

__global__ void kernel(float *data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

int main() {
    printf("=== Occupancy Tuning ===\n\n");

    int n = 1000000;
    size_t size = n * sizeof(float);

    float *h_data = (float*)malloc(size);
    for (int i = 0; i < n; i++) h_data[i] = i;

    float *d_data;
    CUDA_CHECK(cudaMalloc(&d_data, size));
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int threads = 256;
    int blocks = (n + threads - 1) / threads;

    cudaEventRecord(start);
    kernel<<<blocks, threads>>>(d_data, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    CUDA_CHECK(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));

    printf("Processed %d elements in %.2f ms\n", n, ms);
    printf("Bandwidth: %.2f GB/s\n", (size * 2 / 1e9) / (ms / 1000.0));

    free(h_data);
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

## Key Takeaways

1. Occupancy = active warps / maximum warps per SM
2. Balance registers, shared memory, threads per block
3. Use cudaOccupancyMaxPotentialBlockSize
4. Higher occupancy doesn't always mean faster

## Next StepsContinue to: **15_next_topic.ipynb**

## Notes*Use this space to write your own notes and observations:*------